In [42]:
# THE BELOW COMMENTED LINES OF CODE ARE AS A RESULT OF PERFORMING THIS
# PROJECT USING GOOGLE COLAB
#_____________________________________________________________________
#_______________ MOMELEZI DLAVU (DISEASE PREDICTIION) ________________
#_____________________________________________________________________

#from google.colab import files
#uploaded = files.upload()
#for filename in uploaded.keys():
#  print('Train_data.csv', filename)

#from google.colab import files
#uploadd = files.upload()
#for name in uploadd.keys():
#  print('test_set.csv', name)




In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

In [55]:
df = pd.read_csv('Train_data.csv')
test_set = pd.read_csv('test_data.csv')

In [56]:


# Converting the disease column in the training dataset to be binary class (Not Diseased or Diseased)
for index, row in df.iterrows():
    if row['Disease'] == "Healthy":
        df.at[index, 'Disease'] = "Not Diseased"
    else:
        df.at[index, 'Disease'] = "Diseased"

# Converting the disease column in the test dataset to be a binary class (Not Diseased or Diseased)
for index, row in test_set.iterrows():
  if row['Disease'] == "Healthy":
    test_set.at[index, 'Disease'] = "Not Diseased"
  else:
    test_set.at[index, 'Disease'] = "Diseased"

#_____________________________________________________________________________
#_____________________________________________________________________________

df.head()
df.isnull().sum()
df.duplicated()
df.drop_duplicates()

,Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,...,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein,Disease
0,0.739597,0.650198,0.713631,0.868491,0.687433,0.529895,0.290006,0.631045,0.001328,0.795829,...,0.502665,0.215560,0.512941,0.064187,0.610827,0.939485,0.095512,0.465957,0.769230,Not Diseased
1,0.121786,0.023058,0.944893,0.905372,0.507711,0.403033,0.164216,0.307553,0.207938,0.505562,...,0.856810,0.652465,0.106961,0.942549,0.344261,0.666368,0.659060,0.816982,0.401166,Diseased
2,0.452539,0.116135,0.544560,0.400640,0.294538,0.382021,0.625267,0.295122,0.868369,0.026808,...,0.466795,0.387332,0.421763,0.007186,0.506918,0.431704,0.417295,0.799074,0.779208,Diseased
3,0.136609,0.015605,0.419957,0.191487,0.081168,0.166214,0.073293,0.668719,0.125447,0.501051,...,0.016256,0.040137,0.826721,0.265415,0.594148,0.225756,0.490349,0.637061,0.354094,Diseased
4,0.176737,0.752220,0.971779,0.785286,0.443880,0.439851,0.894991,0.442159,0.257288,0.805987,...,0.429431,0.146294,0.221574,0.015280,0.567115,0.841412,0.153350,0.794008,0.094970,Diseased


In [110]:

# Defining the predictor variables in the dataset
X = df[["Glucose", "Cholesterol", "Hemoglobin", "Platelets", "White Blood Cells",
        "Red Blood Cells", "Hematocrit", "Mean Corpuscular Volume", "Mean Corpuscular Hemoglobin",
        "Mean Corpuscular Hemoglobin Concentration", "Insulin", "BMI", "Systolic Blood Pressure",
        "Diastolic Blood Pressure", "Triglycerides", "HbA1c", "LDL Cholesterol", "HDL Cholesterol",
        "ALT", "AST", "Heart Rate", "Creatinine", "Troponin", "C-reactive Protein"]]

X_test_set = test_set[["Glucose", "Cholesterol", "Hemoglobin", "Platelets", "White Blood Cells",
        "Red Blood Cells", "Hematocrit", "Mean Corpuscular Volume", "Mean Corpuscular Hemoglobin",
        "Mean Corpuscular Hemoglobin Concentration", "Insulin", "BMI", "Systolic Blood Pressure",
        "Diastolic Blood Pressure", "Triglycerides", "HbA1c", "LDL Cholesterol", "HDL Cholesterol",
        "ALT", "AST", "Heart Rate", "Creatinine", "Troponin", "C-reactive Protein"]]

# Defining the target varible in the dataset
y = df["Disease"]

y_test_set = test_set["Disease"]

# Spliting the dataset into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [111]:
# Merging the response variable with the rest of the predictor variables
X_train = pd.DataFrame(X_train)
X_train['Disease'] = y_train

In [112]:
# Checking the response outcomes
print(np.unique(y_train))

['Diseased' 'Not Diseased']


In [113]:
# Defining the minority and majority classes of this dataset respectively
train_minority = X_train[X_train['Disease'] == "Not Diseased"]
train_majority = X_train[X_train['Disease'] == "Diseased"]


In [114]:
# Performing upsample method to increase the instances of the minority class (NOT DISEASED)
train_minority_upsampled = resample(train_minority, replace = True,
                                    n_samples = len(train_majority),
                                    random_state = 10)

In [115]:
# Combining the upsampled data with the majority class of this dataset
upsampled_train = pd.concat([train_majority, train_minority_upsampled])


In [116]:
# Seperating the predictor variables from the response variable
X_train = upsampled_train.drop('Disease', axis = 1)
y_train = upsampled_train['Disease']

In [119]:
X_train.head(10)

,Glucose,Cholesterol,Hemoglobin,Platelets,White Blood Cells,Red Blood Cells,Hematocrit,Mean Corpuscular Volume,Mean Corpuscular Hemoglobin,Mean Corpuscular Hemoglobin Concentration,...,Triglycerides,HbA1c,LDL Cholesterol,HDL Cholesterol,ALT,AST,Heart Rate,Creatinine,Troponin,C-reactive Protein
564,0.407101,0.124738,0.983306,0.663867,0.361113,0.663716,0.232516,0.341056,0.847441,0.309766,...,0.299213,0.491185,0.701914,0.218104,0.790341,0.570902,0.339125,0.310964,0.310900,0.622403
297,0.196797,0.165429,0.866889,0.189737,0.844529,0.263589,0.306305,0.150618,0.915391,0.968273,...,0.630936,0.206223,0.798043,0.221106,0.760611,0.591314,0.544136,0.494329,0.012287,0.633088
932,0.722428,0.300282,0.487907,0.200865,0.402035,0.729285,0.851338,0.820546,0.106637,0.006947,...,0.128765,0.022621,0.061317,0.644191,0.715823,0.417170,0.639148,0.213026,0.549920,0.036800
239,0.143772,0.089600,0.027259,0.171121,0.744950,0.452787,0.871778,0.995263,0.872038,0.311175,...,0.012693,0.133001,0.261072,0.200323,0.363050,0.558191,0.304237,0.306539,0.566289,0.754189
99,0.143772,0.089600,0.027259,0.171121,0.744950,0.452787,0.871778,0.995263,0.872038,0.311175,...,0.012693,0.133001,0.261072,0.200323,0.363050,0.558191,0.304237,0.306539,0.566289,0.754189
1461,0.086810,0.271507,0.841366,0.954619,0.412914,0.332666,0.885491,0.991943,0.206693,0.442733,...,0.068169,0.853538,0.293808,0.150089,0.829810,0.236373,0.139593,0.795061,0.772244,0.550695
281,0.353734,0.757757,0.755007,0.012594,0.227684,0.425117,0.387461,0.461418,0.305588,0.741120,...,0.393263,0.446854,0.729376,0.615543,0.794735,0.233890,0.612188,0.407891,0.426863,0.532100
354,0.729868,0.905026,0.520473,0.306955,0.743164,0.667150,0.769573,0.183771,0.246542,0.161920,...,0.184604,0.188792,0.977656,0.341007,0.797158,0.994460,0.591303,0.425296,0.007490,0.196192
1313,0.104427,0.397638,0.747259,0.330296,0.010139,0.904301,0.180428,0.529738,0.829100,0.644510,...,0.271828,0.764430,0.983826,0.421371,0.583997,0.043021,0.610860,0.486840,0.362143,0.004867
2076,0.320138,0.442105,0.331686,0.171060,0.990786,0.992377,0.913721,0.880020,0.025514,0.931498,...,0.141657,0.313360,0.551162,0.922278,0.485114,0.486317,0.519647,0.076333,0.073240,0.084155


In [118]:
y_train.head(10)

564     Diseased
297     Diseased
932     Diseased
239     Diseased
99      Diseased
1461    Diseased
281     Diseased
354     Diseased
1313    Diseased
2076    Diseased
Name: Disease, dtype: object

In [120]:
# Scaling the datasets
scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)
X_test_set = scalar.fit_transform(X_test_set)



In [121]:
# Defining the Support Vector Machine model
svm_model = SVC()

# Defining parameters for tuning
parameters = {
    'kernel' : ['linear', 'poly', 'rbf'],
    'C' : [0.001, 0.01, 0.1, 1, 5],
    'degree' : [2, 3, 4, 5, 6],
    'gamma' : [1, 0.1, 0.01]
}

# Performing the grid search for parameter tuning at 5-fold cross validation
grid_search = GridSearchCV(estimator = svm_model, param_grid = parameters, verbose = 3, cv = 5)
grid_search.fit(X_train, y_train)

# Obtaining the best parameter and score
best_params = grid_search.best_params_
best_score = grid_search.best_score_
print("\n___________________________________________________________")
print("Best Parameter: ", best_params)
print("Best Score: ", best_score)



Fitting 5 folds for each of 225 candidates, totalling 1125 fits
[CV 1/5] END C=0.001, degree=2, gamma=1, kernel=linear;, score=0.926 total time=   0.2s
[CV 2/5] END C=0.001, degree=2, gamma=1, kernel=linear;, score=0.919 total time=   0.2s
[CV 3/5] END C=0.001, degree=2, gamma=1, kernel=linear;, score=0.904 total time=   0.2s
[CV 4/5] END C=0.001, degree=2, gamma=1, kernel=linear;, score=0.932 total time=   0.2s
[CV 5/5] END C=0.001, degree=2, gamma=1, kernel=linear;, score=0.904 total time=   0.2s
[CV 1/5] END C=0.001, degree=2, gamma=1, kernel=poly;, score=1.000 total time=   0.0s
[CV 2/5] END C=0.001, degree=2, gamma=1, kernel=poly;, score=1.000 total time=   0.0s
[CV 3/5] END C=0.001, degree=2, gamma=1, kernel=poly;, score=1.000 total time=   0.0s
[CV 4/5] END C=0.001, degree=2, gamma=1, kernel=poly;, score=1.000 total time=   0.0s
[CV 5/5] END C=0.001, degree=2, gamma=1, kernel=poly;, score=1.000 total time=   0.0s
[CV 1/5] END C=0.001, degree=2, gamma=1, kernel=rbf;, score=0.499 

In [122]:
# Training the Support Vector Machine with the best parameters
best_svm_model = SVC(**best_params, probability = True)

best_svm_model.fit(X_train, y_train)

# Evaluating the models' accuracy
accuracy = best_svm_model.score(X_test_set, y_test_set)
print("___________________________\nAccuracy: ", accuracy)

# Getting the predicted probabilities
y_prob = best_svm_model.predict_proba(X_test_set)

# Displaying the Confusion Matrix and Classification Report
print("\n___________________________\n____ Confusion Matrix _____\n")
predictions = best_svm_model.predict(X_test_set)
print(confusion_matrix(y_test_set, predictions))
print("\n______________________________________________________\n_________________ Classification report ______________\n")
print(classification_report(y_test_set, predictions))




___________________________
Accuracy:  0.7057613168724279

___________________________
____ Confusion Matrix _____

[[342 139]
 [  4   1]]

______________________________________________________
_________________ Classification report ______________

              precision    recall  f1-score   support

    Diseased       0.99      0.71      0.83       481
Not Diseased       0.01      0.20      0.01         5

    accuracy                           0.71       486
   macro avg       0.50      0.46      0.42       486
weighted avg       0.98      0.71      0.82       486



In [86]:
y_test_set.value_counts(normalize=True)


Disease
Diseased        0.989712
Not Diseased    0.010288
Name: proportion, dtype: float64